<a href="https://colab.research.google.com/github/Lornng/FineTune/blob/main/Finetuned_Llama2_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### Text Generation Model

model_id = "Hieu-Pham/Llama2-7B-QLoRA-cooking-text-gen-merged" #@param ["Hieu-Pham/Llama2-7B-QLoRA-cooking-text-gen-merged", "Hieu-Pham/Llama2-7B-IA3-cooking-text-gen-merged", "chloecchng/llama2_merged", "Lornng/Llama2-7b-merged-qlora-cpgQA-text-gen-final"] {type:"string"}
context = "1 small jar chipped beef, cut up 4 boned chicken breasts 1 can cream of mushroom soup 1 carton sour cream 1. Place chipped beef on bottom of baking dish. 2. Place chicken on top of beef. 3. Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\xB0 for 3 hours." #@param {type:"string"}
question = "How much chipped beef is needed?" #@param {type:"string"}

from IPython.display import clear_output
try:
  import torch
  import transformers
except ImportError as e:
  import os, subprocess
  !pip install -q -U bitsandbytes
  !pip install -q -U git+https://github.com/huggingface/transformers.git
  !pip install -q -U git+https://github.com/huggingface/peft.git
  !pip install -q -U git+https://github.com/huggingface/accelerate.git
  !pip install -q datasets einops
  os.mkdir("offload")
clear_output(wait=False)
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, gc

reload_model = False
try:
  model
  reload_model = not (model_name == model_id)
  if reload_model:
    del model
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
except Exception as e:
  reload_model = True

if reload_model:
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"": 0}, offload_folder="/content/offload", torch_dtype=torch.bfloat16)
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  tokenizer.pad_token = tokenizer.eos_token
  clear_output(wait=False)
  model_name = model_id

question = f"{question}\n\n###Answer:\n"
inputs = tokenizer(context, question, return_tensors="pt")
inputs.to("cuda")
input_ids = inputs.input_ids
outputs = model.generate(inputs.input_ids,
                         max_new_tokens=100,
                         do_sample=True,
                         top_k=50,
                         top_p=0.95)
answer = tokenizer.batch_decode(outputs[:, input_ids.shape[1]:])
answer = answer[0].replace('\\n', '\n')
print(answer)

1 small jar chipped beef, cut up

</s>
